In [0]:
import os
import glob
import math
import shutil
import random
import tarfile
import tensorflow as tf
import tensorflow_hub as hub

W0519 03:33:07.798849 140479845271424 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [0]:
random.seed(0)
tf.random.set_random_seed(0)

# Data

This notebook is meant to be run in Google Colab.

In [0]:
try:
  # drive
  from google.colab import drive
  drive.mount('/content/gdrive')
  
  # tensorboard
  !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
  !unzip -o ngrok-stable-linux-amd64.zip
  
  # same as model_dir below
  log_dir = './log'
  get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(log_dir)
  )
  get_ipython().system_raw('./ngrok http 6006 &')
  !curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"
  
except ImportError:
  print('Module google not found, assuming running locally')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
--2019-05-19 03:33:08--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.4.95.48, 52.203.66.95, 3.209.102.29, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.4.95.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16529980 (16M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.26’

ngrok-stable-linux- 100%[===================>]  15.76M  20.1MB/s    in 0.8s    

2019-05-19 03:33:09 (20.1 MB/s) - ‘ngrok-stable-linux-amd64.zip.26’ saved [16529980/16529980]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
https://99e5f3d6.ngrok.io


In [0]:
def maybe_decompress_data(inpath='/content/gdrive/My Drive/indoor-outdoor-classification/data.tar.gz',
                          outpath='/content/data'):
  if not os.path.exists(outpath):
    with tarfile.open(inpath, 'r:gz') as tf:
      tf.extractall()
  else:
    print('{} exists, skipping'.format(outpath))
    
maybe_decompress_data()

/content/data exists, skipping


For cross-validation, we hold out entire videos' frames.

In [0]:
def train_test_split(filenames, class_path_index=-3, n_test_per_class=4):
  train = []
  test = []
  
  classnames = [filename.split(os.sep)[class_path_index] for filename in filenames]
  videonames = [filename.split(os.sep)[-1].split('-')[0] for filename in filenames]
  
  for name in set(classnames):
    videonames_this_class = set([videoname for classname, videoname in zip(classnames, videonames) if classname == name])
    videonames_test = random.sample(videonames_this_class, n_test_per_class)
    videonames_train = videonames_this_class - set(videonames_test)
    train += [filename for filename, videoname in zip(filenames, videonames) if videoname in videonames_train]
    test += [filename for filename, videoname in zip(filenames, videonames) if videoname in videonames_test]
    
  return train, test

train, test = train_test_split(glob.glob('/content/data/**/*.png', recursive=True))

# Model

We start with ResNet_v2_50 and train a binary classifier on top.

In [0]:
params = {
    'class_path_index': -3,
    'batch_size': 128,
    'resize_image_size': 448,
    'crop_image_size': 224,
    'learning_rate': 0.001,
    'num_epochs': 50
}

def model_fn(features, labels, mode, params):
  '''
  Model function (ResNet_v2_50) transfer learning
  '''
  learning_rate = params['learning_rate']
 
  module = hub.Module('https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/3')
  features = module(features)
  
  logits = tf.layers.dense(features, 1, kernel_initializer=tf.initializers.glorot_uniform())
  scores = tf.nn.sigmoid(logits)
  
  if mode == tf.estimator.ModeKeys.PREDICT:
    outputs = {'scores': scores}
    predict_outputs = tf.estimator.export.PredictOutput(outputs)
    export_outputs = {tf.saved_model.signature_constants.PREDICT_METHOD_NAME: predict_outputs}
    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions={'scores': scores},
        export_outputs=export_outputs
    )
  
  predictions = tf.round(scores)
  accuracy = tf.metrics.accuracy(labels, predictions)
  tf.summary.scalar('accuracy', accuracy[1])
  
  nll = tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.cast(labels, tf.float32),
                                                logits=tf.squeeze(logits))
  loss = tf.reduce_mean(nll)
  
  optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)                                    
  train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
  
  return tf.estimator.EstimatorSpec(
      mode=mode,
      loss=loss,
      train_op=train_op,
      predictions={'scores': scores},
      eval_metric_ops={'accuracy': accuracy}
  )
  

def load_image(filename, params):
  '''
  Load and resize an image. 
  For simplicitly, we will resize it without preserving aspect ratio
  '''
  resize_image_size = params['resize_image_size']
  
  buffer = tf.read_file(filename)
  image = tf.image.decode_png(buffer, channels=3)
  image = tf.cast(image, tf.float32)
  image = tf.image.resize_images(image, (resize_image_size, resize_image_size))

  return image

def preprocess_image(image, mode, params):
  '''
  For training, take a random crop and randomly flip left-to-right. 
  For testuation, take a center crop.
  '''
  crop_image_size = params['crop_image_size']
  resize_image_size = params['resize_image_size']

  if mode == tf.estimator.ModeKeys.TRAIN:
    image = tf.image.random_crop(image, (crop_image_size, crop_image_size, 3))
    image = tf.image.random_flip_left_right(image)
  else:
    image = tf.image.central_crop(image, crop_image_size/resize_image_size)
    
  return image
  
def _input_fn(filenames, mode, params):
  '''
  Async input pipeline. Labels are sorted so indoor=0, outdoor=1
  '''
  class_path_index = params['class_path_index']
  batch_size = params['batch_size']
  
  classnames = [filename.split(os.sep)[class_path_index] for filename in filenames]
  lookup = {v: k for k, v in enumerate(sorted(set(classnames)))}
  labels = [lookup[classname] for classname in classnames]
  
  dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
  
  if mode == tf.estimator.ModeKeys.TRAIN:
    dataset = dataset.shuffle(buffer_size=len(filenames))
    dataset = dataset.repeat()
    
  dataset = dataset.map(
      lambda filename, label: (load_image(filename, params), label),
      num_parallel_calls=os.cpu_count()
  )
  dataset = dataset.map(
      lambda filename, label: (preprocess_image(filename, mode, params), label),
      num_parallel_calls=os.cpu_count()
  )
 
  dataset = dataset.batch(batch_size)
  dataset = dataset.prefetch(1)

  return dataset
  
def train_input_fn(filenames, params):
  return _input_fn(filenames, tf.estimator.ModeKeys.TRAIN, params)

def eval_input_fn(filenames, params):
  return _input_fn(filenames, tf.estimator.ModeKeys.EVAL, params)

def serving_input_receiver_fn(params):
  '''
  Add preprocessing to serving graph
  '''
  inputs = tf.placeholder(dtype=tf.float32, shape=[None, None, None, 3])
  image = preprocess_image(inputs, tf.estimator.ModeKeys.PREDICT, params)
  receivers = {tf.saved_model.signature_constants.PREDICT_INPUTS: inputs}
  return tf.estimator.export.TensorServingInputReceiver(image, receivers)

model_dir = 'log'
shutil.rmtree(model_dir, ignore_errors=True)

epoch_steps = math.ceil(len(train)/params['batch_size'])
max_steps = epoch_steps * params['num_epochs']

config = tf.estimator.RunConfig(
    save_checkpoints_steps=epoch_steps
)

estimator = tf.estimator.Estimator(
    model_fn=model_fn,
    params=params,
    model_dir=model_dir,
    config=config
)

train_spec = tf.estimator.TrainSpec(
    input_fn=lambda: train_input_fn(train, params),
    max_steps=max_steps
)

eval_spec = tf.estimator.EvalSpec(
    input_fn=lambda: eval_input_fn(test, params),
    name='test',
    throttle_secs=0
)

tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

for filename, split in zip(['images_train.txt', 'images_test.txt'], [train, test]):
  with open(os.path.join(model_dir, filename), 'w') as f:
    f.write('\n'.join(split))

INFO:tensorflow:Using config: {'_model_dir': 'log', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 16, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc39d0017b8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0519 03:33:12.453240 140479845271424 estimator.py:201] Using config: {'_model_dir': 'log', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 16, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc39d0017b8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Not using Distribute Coordinator.


I0519 03:33:12.468725 140479845271424 estimator_training.py:185] Not using Distribute Coordinator.


INFO:tensorflow:Running training and evaluation locally (non-distributed).


...

I0519 04:11:26.396640 140479845271424 estimator.py:1979] Saving dict for global step 800: accuracy = 0.705, global_step = 800, loss = 3.4521003


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 800: log/model.ckpt-800


I0519 04:11:26.413319 140479845271424 estimator.py:2039] Saving 'checkpoint_path' summary for global step 800: log/model.ckpt-800


INFO:tensorflow:Loss for final step: 0.45407617.


I0519 04:11:26.525375 140479845271424 estimator.py:359] Loss for final step: 0.45407617.


In [0]:
estimator.export_savedmodel(model_dir,
                            lambda: serving_input_receiver_fn(params),
                            strip_default_attrs=True)

Instructions for updating:
Use tf.cast instead.


W0519 04:11:26.565722 140479845271424 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/image_ops_impl.py:618: to_double (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


INFO:tensorflow:Calling model_fn.


I0519 04:11:26.590010 140479845271424 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:No assets to write.


I0519 04:11:31.297927 140479845271424 builder_impl.py:449] No assets to write.


INFO:tensorflow:SavedModel written to: log/temp-b'1558239086'/saved_model.pb


I0519 04:11:32.323876 140479845271424 builder_impl.py:414] SavedModel written to: log/temp-b'1558239086'/saved_model.pb


b'log/1558239086'

In [0]:
def compress_model_dir(inpath=model_dir, 
                       outpath='/content/gdrive/My Drive/indoor-outdoor-classification/{}.tar.gz'.format(model_dir)):
  with tarfile.open(outpath, 'w:gz') as tf:
    tf.add(inpath)
    
compress_model_dir()